<a href="https://colab.research.google.com/github/hamzafarooq/maven-blps-2/blob/main/Module-4/semantic_search_module_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [24]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cuda


Read the data for Paris Hotels

In [10]:
import pandas as pd
df = pd.read_csv("/workspace/maven-blps-2/data/miami-hotel-reviews.csv")

In [11]:
df.shape

(1713, 9)

In [12]:
df.name.value_counts()

name
Sentral Wynwood                                 10
Travelodge by Wyndham Miami Biscayne Bay        10
Miami Princess Hotel                            10
Caoba                                           10
citizenM Miami Worldcenter                      10
                                                ..
The Club Design Condos                           1
Rooms in Kendall Miami                           1
Mimo Home Studio With Private Entrance & Key     1
City Inn                                         1
Mert Lodge                                       1
Name: count, Length: 223, dtype: int64

In [13]:
df = df.drop_duplicates()

In [14]:
df.shape

(1713, 9)

In [15]:
df.name.value_counts()

name
Sentral Wynwood                                 10
Travelodge by Wyndham Miami Biscayne Bay        10
Miami Princess Hotel                            10
Caoba                                           10
citizenM Miami Worldcenter                      10
                                                ..
The Club Design Condos                           1
Rooms in Kendall Miami                           1
Mimo Home Studio With Private Entrance & Key     1
City Inn                                         1
Mert Lodge                                       1
Name: count, Length: 223, dtype: int64

In [16]:
df.head()

,name,category,address,description,score,rank_str,review,review_count,url
0,Sentral Wynwood,hotels,"51 NW 26th St, Miami, FL 33127",Sentral is transforming the travel experience ...,4.5,#60 of 155 hotels in Miami,"I loved it here! Close to restaurants, shoppin...",87,https://www.tripadvisor.com//Hotel_Review-g344...
1,The Guild Downtown | X Miami,condos,"230 NE 4th St, Miami, FL 33132-2231",The Guild is the hotel that combines the comfo...,4.0,#3 of 28 condos in Miami,I also received that email yesterday. Was ver...,159,https://www.tripadvisor.com//Hotel_Review-g344...
2,Beautiful Spanish Residence in Coconut Grove,limited service properties,"1757 Wa Kee NA Dr, Miami, FL 33133-2437","No se que tiene este lugar ,pero me siento com...",4.5,#7 of 18 limited service properties in Miami,What a gorgeous find! Our family of 5 stayed f...,4,https://www.tripadvisor.com//Hotel_Review-g344...
3,"EVEN Hotel Miami - Airport, an IHG Hotel",hotels,"3499 N.W. 25th Street, Miami, FL 33142",Modern Pet Friendly Hotel near Miami Internati...,4.0,#34 of 155 hotels in Miami,Everyone was very nice and helpful. The room ...,461,https://www.tripadvisor.com//Hotel_Review-g344...
4,Mr. C Miami Coconut Grove,hotels,"2988 McFarlane Rd, Miami, FL 33133-6011",From the illustrious fourth generation hospita...,4.5,#13 of 155 hotels in Miami,Relatively small hotel with 5-star service and...,413,https://www.tripadvisor.com//Hotel_Review-g344...


In [18]:

df["combined"] = (
    "name: " + df.name.str.strip()+ "; description: " + df.description.str.strip()
    # +"; desc: "+ df.text.str.strip()
)

In [19]:
import re

df_combined = df.copy()

df_combined["combined"] = df_combined["combined"].apply(lambda x: re.sub("[^a-zA-z0-9\s]", "", str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined["combined"] = df_combined["combined"].apply(lambda x: lower_case(x))

In [77]:
embedder = SentenceTransformer("all-mpnet-base-v2").to(device)

In [78]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

In [79]:
df_combined.head()

,name,category,address,description,score,rank_str,review,review_count,url,combined,embedding
0,Sentral Wynwood,hotels,"51 NW 26th St, Miami, FL 33127",Sentral is transforming the travel experience ...,4.5,#60 of 155 hotels in Miami,"I loved it here! Close to restaurants, shoppin...",87,https://www.tripadvisor.com//Hotel_Review-g344...,name sentral wynwood description sentral is tr...,"[-0.021624146, -0.013639651, -0.027944447, -0...."
1,The Guild Downtown | X Miami,condos,"230 NE 4th St, Miami, FL 33132-2231",The Guild is the hotel that combines the comfo...,4.0,#3 of 28 condos in Miami,I also received that email yesterday. Was ver...,159,https://www.tripadvisor.com//Hotel_Review-g344...,name the guild downtown x miami description t...,"[-0.009419132, -0.00914356, -0.015006056, 0.01..."
2,Beautiful Spanish Residence in Coconut Grove,limited service properties,"1757 Wa Kee NA Dr, Miami, FL 33133-2437","No se que tiene este lugar ,pero me siento com...",4.5,#7 of 18 limited service properties in Miami,What a gorgeous find! Our family of 5 stayed f...,4,https://www.tripadvisor.com//Hotel_Review-g344...,name beautiful spanish residence in coconut gr...,"[-0.039776742, 0.0056628343, -0.026097713, -0...."
3,"EVEN Hotel Miami - Airport, an IHG Hotel",hotels,"3499 N.W. 25th Street, Miami, FL 33142",Modern Pet Friendly Hotel near Miami Internati...,4.0,#34 of 155 hotels in Miami,Everyone was very nice and helpful. The room ...,461,https://www.tripadvisor.com//Hotel_Review-g344...,name even hotel miami airport an ihg hotel de...,"[-0.016368823, 0.011991538, -0.002208242, 0.01..."
4,Mr. C Miami Coconut Grove,hotels,"2988 McFarlane Rd, Miami, FL 33133-6011",From the illustrious fourth generation hospita...,4.5,#13 of 155 hotels in Miami,Relatively small hotel with 5-star service and...,413,https://www.tripadvisor.com//Hotel_Review-g344...,name mr c miami coconut grove description from...,"[-0.0006249037, -0.005485512, 0.002435267, -0...."


In [80]:
sample = df_combined[:10].copy()

In [81]:
%%time

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

CPU times: user 47.1 s, sys: 0 ns, total: 47.1 s
Wall time: 6.16 s


In [38]:
sample["embedding"][0].shape

(1024,)

In [31]:
sample.to_pickle("/workspace/maven-blps-2/Module 4/df.pkl")    #to save the dataframe, df to 123.pkl

In [32]:
df_with_embedding = pd.read_pickle("/workspace/maven-blps-2/Module 4/df.pkl") #to load 123.pkl back to the dataframe df

In [33]:
query = "close to Hotel Malte"

query_embedding = embedder.encode(query, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
df_with_embedding.combined[0]

'name sentral wynwood description sentral is transforming the travel experience and changing the way you stay at sentral wynwoodyoull enjoy spacious apartmentstyle rooms for you and your group to create new memories all under one roof theyre designed to feel like home just better youll be located right inside miamis vibrant cultural epicenter the citys coolest neighborhood between visits to muchloved art museums top restaurants and outdoor marketsyoull enjoy a fully stocked room and luxury amenities including a stunning rooftop pool fitness center and lounge area with 247 service from our friendly staff youll have everything you needplus so much more to bring this space to life our inhouse creative team designed original wallpapers and prints inspired by wynwoods impressive art culturewith a touch of sentral thrown in the mix'

In [36]:
corpus_embeddings = embedder.encode(df_with_embedding.combined, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
df_with_embedding["similarity"] = df_with_embedding.embedding.apply(lambda x: cosine_similarity(x, query_embedding))

In [43]:
df_with_embedding.sort_values("similarity", ascending=False)

,name,category,address,description,score,rank_str,review,review_count,url,combined,embedding,similarity
6,Sinbad Motel,hotels,"6150 Biscayne Blvd, Miami, FL 33137-2227",Finding an ideal budget friendly hotel in Miam...,3.5,#96 of 155 hotels in Miami,This place is totally disgusting! Do not stay ...,109,https://www.tripadvisor.com//Hotel_Review-g344...,name sinbad motel description finding an ideal...,"[-0.011723803, -0.00045236462, -0.00030043398,...",0.826755
3,"EVEN Hotel Miami - Airport, an IHG Hotel",hotels,"3499 N.W. 25th Street, Miami, FL 33142",Modern Pet Friendly Hotel near Miami Internati...,4.0,#34 of 155 hotels in Miami,Everyone was very nice and helpful. The room ...,461,https://www.tripadvisor.com//Hotel_Review-g344...,name even hotel miami airport an ihg hotel de...,"[-0.016368823, 0.011991538, -0.002208242, 0.01...",0.826531
9,Mint House Miami - Downtown,apartment hotel,"340 SE 3rd St, Miami, FL 33131",Make The Magic City your own personal paradise...,4.0,#1 of 1 apartment hotel in Miami,"The apartment was amazing, but I received no c...",50,https://www.tripadvisor.com//Hotel_Review-g344...,name mint house miami downtown description ma...,"[-0.032159954, -0.012157241, -0.0114207165, -0...",0.819921
7,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,The place was so bad we couldn't stay there. T...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.005738122, 0.0037997835, 0.0045639914, -0....",0.819680
0,Sentral Wynwood,hotels,"51 NW 26th St, Miami, FL 33127",Sentral is transforming the travel experience ...,4.5,#60 of 155 hotels in Miami,"I loved it here! Close to restaurants, shoppin...",87,https://www.tripadvisor.com//Hotel_Review-g344...,name sentral wynwood description sentral is tr...,"[-0.021624146, -0.013639651, -0.027944447, -0....",0.816665
5,Atwell Suites Miami Brickell,hotels,"145 SW 11th St, Miami, FL 33130-4005","Save yourself the time and money, don’t go the...",3.5,#109 of 155 hotels in Miami,The room was great and bed comfy But sadly so...,16,https://www.tripadvisor.com//Hotel_Review-g344...,name atwell suites miami brickell description ...,"[-0.010118131, -0.014490537, -0.011030351, -0....",0.815330
1,The Guild Downtown | X Miami,condos,"230 NE 4th St, Miami, FL 33132-2231",The Guild is the hotel that combines the comfo...,4.0,#3 of 28 condos in Miami,I also received that email yesterday. Was ver...,159,https://www.tripadvisor.com//Hotel_Review-g344...,name the guild downtown x miami description t...,"[-0.009419132, -0.00914356, -0.015006056, 0.01...",0.813040
8,Courtyard by Marriott Miami Airport,hotels,"1201 NW Le Jeune Road, Miami, FL 33126",Courtyard Miami Airport is a pet-friendly hote...,4.0,#56 of 155 hotels in Miami,The proximity to the Miami airport and free sh...,1289,https://www.tripadvisor.com//Hotel_Review-g344...,name courtyard by marriott miami airport descr...,"[-0.022358036, -0.00310383, -0.0029010912, 0.0...",0.811355
4,Mr. C Miami Coconut Grove,hotels,"2988 McFarlane Rd, Miami, FL 33133-6011",From the illustrious fourth generation hospita...,4.5,#13 of 155 hotels in Miami,Relatively small hotel with 5-star service and...,413,https://www.tripadvisor.com//Hotel_Review-g344...,name mr c miami coconut grove description from...,"[-0.0006249037, -0.005485512, 0.002435267, -0....",0.802573
2,Beautiful Spanish Residence in Coconut Grove,limited service properties,"1757 Wa Kee NA Dr, Miami, FL 33133-2437","No se que tiene este lugar ,pero me siento com...",4.5,#7 of 18 limited service properties in Miami,What a gorgeous find! Our family of 5 stayed f...,4,https://www.tripadvisor.com//Hotel_Review-g344...,name beautiful spanish residence in coconut gr...,"[-0.039776742, 0.0056628343, -0.026097713, -0....",0.791900


# Entire Data

In [82]:
%%time

df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))


CPU times: user 1h 3min 56s, sys: 2.18 s, total: 1h 3min 58s
Wall time: 8min 21s


In [83]:
df_combined.to_pickle("/workspace/maven-blps-2/Module 4/entire_data.pkl")

## Embedding upload

In [84]:
import pandas as pd
df = pd.read_pickle('/workspace/maven-blps-2/Module 4/entire_data.pkl') #to load 123.pkl back to the dataframe df

In [85]:
df.head()

,name,category,address,description,score,rank_str,review,review_count,url,combined,embedding
0,Sentral Wynwood,hotels,"51 NW 26th St, Miami, FL 33127",Sentral is transforming the travel experience ...,4.5,#60 of 155 hotels in Miami,"I loved it here! Close to restaurants, shoppin...",87,https://www.tripadvisor.com//Hotel_Review-g344...,name sentral wynwood description sentral is tr...,"[-0.0083673, 0.019167796, -0.0017852152, 0.040..."
1,The Guild Downtown | X Miami,condos,"230 NE 4th St, Miami, FL 33132-2231",The Guild is the hotel that combines the comfo...,4.0,#3 of 28 condos in Miami,I also received that email yesterday. Was ver...,159,https://www.tripadvisor.com//Hotel_Review-g344...,name the guild downtown x miami description t...,"[-0.040207338, 0.055548247, 0.017145013, 0.059..."
2,Beautiful Spanish Residence in Coconut Grove,limited service properties,"1757 Wa Kee NA Dr, Miami, FL 33133-2437","No se que tiene este lugar ,pero me siento com...",4.5,#7 of 18 limited service properties in Miami,What a gorgeous find! Our family of 5 stayed f...,4,https://www.tripadvisor.com//Hotel_Review-g344...,name beautiful spanish residence in coconut gr...,"[-0.040465526, 0.025397982, -0.028705088, 0.01..."
3,"EVEN Hotel Miami - Airport, an IHG Hotel",hotels,"3499 N.W. 25th Street, Miami, FL 33142",Modern Pet Friendly Hotel near Miami Internati...,4.0,#34 of 155 hotels in Miami,Everyone was very nice and helpful. The room ...,461,https://www.tripadvisor.com//Hotel_Review-g344...,name even hotel miami airport an ihg hotel de...,"[-0.019085515, 0.0014932863, 0.010218448, 0.03..."
4,Mr. C Miami Coconut Grove,hotels,"2988 McFarlane Rd, Miami, FL 33133-6011",From the illustrious fourth generation hospita...,4.5,#13 of 155 hotels in Miami,Relatively small hotel with 5-star service and...,413,https://www.tripadvisor.com//Hotel_Review-g344...,name mr c miami coconut grove description from...,"[-0.0067784884, -0.011433906, -0.010785466, 0...."


In [86]:
# search through the reviews for a specific product
def search_reviews(df, query, n=5, pprint=True):
    query_embedding = embedder.encode(query, show_progress_bar=True)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
    )

    return results

In [87]:
query = "close to Louvre and great food nearby but not too expensive"

In [88]:
results = search_reviews(df, query, 10)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [89]:
results

,name,category,address,description,score,rank_str,review,review_count,url,combined,embedding,similarity
605,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,"Acceptable low budget hotel, but if you go wit...",283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
118,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,This is a three story apartment building with ...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
1027,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,"This hotel is nasty, the excuse is the renovat...",283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
1144,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,Very good experience. The staff are friendly ...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
7,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,The place was so bad we couldn't stay there. T...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
1385,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,The units are still in the process of being fi...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
644,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,They were callous and unresponsive to my numer...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
12,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,"Although the building is older, you can see th...",283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
1340,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,I was charged through Expedia upon booking in ...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158
1212,Fortuna House Apartments,condos,"432 NE 26th St, Miami, FL 33137-4792","Acceptable low budget hotel, but if you go wit...",3.5,#9 of 28 condos in Miami,I enjoyed the Check in .The young lady at the ...,283,https://www.tripadvisor.com//Hotel_Review-g344...,name fortuna house apartments description acce...,"[-0.017952722, -0.080992386, 0.008179171, 0.06...",0.434158


In [90]:
# hlist = []
# for r in results.index:
#     if results.name[r] not in hlist:
#         smalldf = results.loc[results.name == results.name[r]]
#         print('Hotel name: {}'.format(results.name[r]))
#         print('Price per night: {}'.format(results.price_per_night.max()))
#         print('Description: {}'.format(results.description[r]))
#         print('Similarity Score: {}'.format(smalldf.similarity.max()))
#         print('Rating Score: {}'.format(smalldf.rating.max()))

#         print('\n')
#         print('Relevant Reviews:')
#         for s in smalldf.index:
#             print(smalldf.text[s])
#             print('\n')
#         print('xxx---xxx')
#         print('\n')
#         hlist.append(results.name[r])


In [ ]:
# df_combined.shape[1]

In [ ]:
# df = df_combined.copy()

Take all the reviews which are closest to the query, and groupby the hotel name

In [91]:
def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))

  results = df.sort_values("similarity", ascending=False).head(n)

  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name": results.name[r],
            "description": results.description[r],
            "score": smalldf.similarity[r],
            # "rating": smalldf.rating.max(),
            "relevant_reviews": [ smalldf.review[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist

In [92]:
search('near Eiffel')

[{'name': 'SLS LUX Brickell',
  'description': 'Just steps away from Brickell City Centre, SLS LUX Brickell offers guests ultimate access to an oasis of exclusive privileges. Sophisticated interiors by design duo Yabu Pushelberg, original works of art by contemporary masters like Fernando Botero and Fabian Burgos and lux garden level pool terrace designed to inspire style terrace designed to inspire style beyond compare.',
  'score': 0.39490873,
  'relevant_reviews': ['Kervins was extremely helpful in helping my family get to know the hotel. All of the staff has been very accommodating and it has been a great experience. The room is wonderful and very clean. I recommend !',
   'Got shown around the room by Kursens Dor, who also carried all our luggage there. He had great customer service and was super friendly. The room is very large and has a luxurous feeling. We are very happy with our visit to the hotel and would go again.',
   'Always a great stay. Staff is so helpful and hotel is 

# Building the API

In [93]:

import gradio as gr

def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))

  results = df.sort_values("similarity", ascending=False).head(n)

  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          smallarr = smalldf.similarity[r].max()
          # sm = smalldf.rating[r].mean()

          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name": results.name[r],
            "description": results.description[r],
            "relevance score": smallarr.tolist(),
            # "rating": sm.tolist(),
            "relevant_reviews": [smalldf.review[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])

  return resultlist

# def greet(query):
#     bm25 = search(query)
#     return bm25

# demo = gr.Interface(fn=greet, inputs="text", outputs="json")

# demo.launch(share=True)

In [96]:
search("near Art Deco Historic District")

[{'name': 'Courtyard Miami Downtown / Brickell Area',
  'description': 'Finding an ideal family-friendly hotel in Miami does not have to be difficult. Welcome to Courtyard Miami Downtown/Brickell Area, a nice option for travelers like you.For those interested in checking out popular landmarks while visiting Miami, Miami Courtyard is located a short distance from Gesu Catholic Church (0.3 mi) and Port of Miami (0.9 mi).You’ll enjoy relaxing rooms that offer a flat screen TV and air conditioning, and you can stay connected during your stay as Courtyard Miami offers guests free wifi.The hotel features a 24 hour front desk, a concierge, and room service. Plus, Miami Courtyard offers a pool and a lounge, providing a pleasant respite from your busy day. For guests with a vehicle, parking is available.While in Miami be sure to experience local ribs favorites at Nusr-Et, Toro Toro Restaurant & Bar, or Hard Rock Cafe.Should time allow, Bayside Marketplace, American Airlines Arena, and Phillip a